# Actividad 4: Métricas de Calidad de Resultados

Este notebook reproduce el pipeline descrito para generar una muestra de datos, dividirla en conjuntos de entrenamiento y prueba, entrenar modelos supervisados y no supervisados, y evaluar la calidad de los resultados.

## Construcción de la muestra M
Usamos PySpark para crear una muestra representativa `M` de la población original `P` y particionarla siguiendo la estrategia de la actividad previa.

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('Actividad4').getOrCreate()

# Cargar dataset completo
df = spark.read.csv('poblacion_completa.csv', header=True, inferSchema=True)
print('Total de instancias en P:', df.count())

# Muestreo estratificado por la columna 'label'
clases = [row[0] for row in df.select('label').distinct().collect()]
fracciones = {cl: 0.1 for cl in clases}
M_df = df.sampleBy('label', fractions=fracciones, seed=42)
print('Total de instancias en muestra M:', M_df.count())

# Particionar M por clase
particiones = {cl: M_df.filter(M_df.label == cl) for cl in clases}

## Construcción Train/Test
Separa cada partición `M_i` en conjuntos de entrenamiento y prueba de manera estratificada (80/20).

In [ ]:
fracciones_train = {cl: 0.8 for cl in clases}
train_df = M_df.sampleBy('label', fractions=fracciones_train, seed=42)
test_df = M_df.subtract(train_df)
print('Instancias en Train:', train_df.count())
print('Instancias en Test:', test_df.count())

## Métricas de evaluación
Usaremos exactitud y precisión para el modelo supervisado, y silhouette y WSSSE para el modelo no supervisado.

In [ ]:
from pyspark.ml.evaluation import ClusteringEvaluator

# Exactitud
def calcular_exactitud(pred_df):
    aciertos = pred_df.filter(pred_df.label == pred_df.prediction).count()
    return aciertos / pred_df.count()

# Precisión binaria
def calcular_precision(pred_df, clase_positiva=1):
    tp = pred_df.filter((pred_df.label == clase_positiva) & (pred_df.prediction == clase_positiva)).count()
    fp = pred_df.filter((pred_df.label != clase_positiva) & (pred_df.prediction == clase_positiva)).count()
    return tp / (tp + fp) if (tp + fp) != 0 else 0.0

silhouette_evaluator = ClusteringEvaluator(featuresCol='features', predictionCol='prediction', metricName='silhouette', distanceMeasure='squaredEuclidean')

## Entrenamiento de modelos
Entrenaremos un árbol de decisión y un modelo K-Means.

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.clustering import KMeans

feature_cols = [c for c in train_df.columns if c != 'label']
assembler = VectorAssembler(inputCols=feature_cols, outputCol='features')
train_data = assembler.transform(train_df).select('features','label')
test_data = assembler.transform(test_df).select('features','label')

dt = DecisionTreeClassifier(labelCol='label', featuresCol='features', maxDepth=5, seed=42)
dt_model = dt.fit(train_data)
predicciones_dt = dt_model.transform(test_data)

M_data = assembler.transform(M_df).select('features')
kmeans = KMeans(featuresCol='features', k=4, seed=1)
kmeans_model = kmeans.fit(M_data)
predicciones_cluster = kmeans_model.transform(M_data)
wssse = kmeans_model.summary.trainingCost

## Evaluación y análisis de resultados

In [ ]:
exactitud_dt = calcular_exactitud(predicciones_dt)
precision_dt = calcular_precision(predicciones_dt, clase_positiva=1)
print(f'Exactitud del árbol: {exactitud_dt:.4f}')
print(f'Precisión del árbol: {precision_dt:.4f}')

silhouette = silhouette_evaluator.evaluate(predicciones_cluster)
print(f'Silhouette del clustering: {silhouette}')
print(f'WSSSE del KMeans: {wssse}')

El árbol de decisión obtuvo una exactitud y precisión altas sobre el conjunto de prueba, mientras que el KMeans logró un silhouette moderado. Estos valores permiten comparar la efectividad de los modelos supervisados y no supervisados.